### Install

In [1]:
!pip install pycaret[full]
!pip install jinja2
!pip install lightgbm
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import

In [32]:
from pycaret import regression

from xgboost import XGBRegressor

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, RobustScaler, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import joblib

### Prepare the data

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/220919/fifa_pay_prediction/03_work_code/Querkowe

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/220919/fifa_pay_prediction/03_work_code/Querkowe


In [4]:
train = pd.read_csv('../../00_original_data/FIFA_train.csv').copy()

#### Preprocess

In [5]:
#contract_until 변수 int 형으로 변환
# 계약 연도만 추출
def func(string:object) -> int:
    """계약 연도만 추출하여 int로 반환"""
    string = string[-4:]
    return int(string)


train['contract_until'] = train['contract_until'].apply(func)

##### 열 추가

1. stat_potential - stat_overall | RMSE: 446
1. (stat_potential - stat_overall) * stat_potential | 'RMSE': 431
1. (stat_potential - stat_overall) * stat_overall | 'RMSE': 426
1. (stat_potential - stat_overall) * age | 'RMSE': 434
1. stat_potential^2 - stat_overall^2 | 'RMSE': 427
1. stat_potential^2 / stat_overall^2 | 'RMSE': 440
1. stat_potential \* stat_overall * age | RMSE: 421
1. stat_potential^2 / stat_overall / age | 'RMSE': 438
1. stat_potential^2 / stat_overall^2 | 'RMSE': 439
1. stat_potential / stat_overall | 'RMSE': 440
1. (stat_potential^2 - stat_overall^2) * age | 'RMSE': 435

In [6]:
# # 1
# train['stat_gap'] = train['stat_potential'] - train['stat_overall']
# train['stat_gap']

In [7]:
# # 2
# train['stat_gap'] = (train['stat_potential'] - train['stat_overall']) * train['stat_potential']
# train['stat_gap']

In [8]:
# 3
# train['stat_gap'] = (train['stat_potential'] - train['stat_overall']) * train['stat_overall']
# train['stat_gap']

In [9]:
# # 4
# train['stat_gap'] = (train['stat_potential'] - train['stat_overall']) *  train['age']
# train['stat_gap']

In [10]:
# # 5
# train['stat_gap'] = train['stat_potential'] ** 2 - train['stat_overall'] ** 2
# train['stat_gap']

In [11]:
# # 6
# train['stat_gap'] = train['stat_potential'].astype(float) ** 2 / train['stat_overall'].astype(float) ** 2
# train['stat_gap']

In [12]:
# # 7
train['stat_poa'] = train['stat_potential'] * train['stat_overall'] * train['age']
# train['stat_poa']

In [13]:
# # 8
# train['stat_pp_oa'] = (train['stat_potential'] ** 2) / train['stat_overall'] / train['age']

In [14]:
# 9
# train['stat_ppa_o'] = (train['stat_potential'] ** 2) * train['age'] / train['stat_overall']

In [15]:
# 11
# train['stat_pp_oo_a'] = (train['stat_potential'] ** 2 - train['stat_overall'] ** 2) *  train['age']

##### Encoding & Scaling

In [16]:
# 문자형 변수 label 인코딩 진행
train['continent'] = train['continent'].astype('category').cat.codes
train['position'] = train['position'].astype('category').cat.codes
train['stat_skill_moves'] = train['stat_skill_moves'].astype('category').cat.codes

In [17]:
# 숫자형 변수 scaling 진행

minmax_features = ['reputation', 'stat_overall', 'stat_potential', 'contract_until'] # , 'stat_skill_moves'
minmax_transformer = MinMaxScaler() #  # RobustScaler()

standard_features = ['age', 'stat_poa']
standard_transformer = StandardScaler()

train[minmax_features] = minmax_transformer.fit_transform(train[minmax_features])
train[standard_features] = standard_transformer.fit_transform(train[standard_features])
# train['value'] = train['value'] / 1000

### Set up PyCaret Regression Model

In [18]:
fe_dict = {}

In [19]:
pycaret_model = regression.setup(data=train[train.keys().drop(['id', 'name'])], 
              target='value', 
              session_id=42) # Random seed 

,Description,Value
0,session_id,42
1,Target,value
2,Original Data,"(8932, 11)"
3,Missing Values,False
4,Numeric Features,9
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(6252, 10)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[], target='value',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
              

### Catboost

In [20]:
ctb = regression.create_model('catboost')
ctb_tuned = regression.tune_model(ctb, optimize='RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,264257.6701,3.741972e+11,6.117166e+05,0.9880,0.3770,0.1949
1,264330.4534,5.967101e+11,7.724701e+05,0.9794,0.3375,0.2794
2,249405.0548,4.138453e+11,6.433081e+05,0.9818,0.4684,0.2548
3,266698.0986,4.666040e+11,6.830842e+05,0.9824,0.3602,0.2650
4,234563.1068,2.916387e+11,5.400359e+05,0.9926,0.3810,0.2141
5,232917.9810,2.290546e+11,4.785964e+05,0.9916,0.4446,0.3656
6,326700.9477,1.042732e+12,1.021142e+06,0.9775,0.4068,0.2475
7,300795.1921,6.081909e+11,7.798659e+05,0.9876,0.4077,0.2778
8,236865.5961,3.117629e+11,5.583574e+05,0.9872,0.4923,0.2651


INFO:logs:create_model_container: 2
INFO:logs:master_model_container: 2
INFO:logs:display_container: 3
INFO:logs:<catboost.core.CatBoostRegressor object at 0x7f7a1a5a42d0>
INFO:logs:tune_model() succesfully completed......................................


In [21]:
# 결과 모델 점수

ctb_tuned.get_best_score()['learn']['RMSE']

421487.8915681472

In [22]:
print('feature impotance')
print("".ljust(44, '='))
for i in range(len(ctb_tuned.feature_names_)):
  print(ctb_tuned.feature_names_[i].ljust(20), ':', ctb_tuned.feature_importances_[i])

feature impotance
age                  : 3.002627401238699
continent            : 0.009936844890739458
contract_until       : 0.015611543287944844
position             : 1.0124965202495713
reputation           : 0.00041597766822851613
stat_overall         : 74.43943577671315
stat_potential       : 18.642207543531015
stat_skill_moves     : 0.8769616529825052
stat_poa             : 1.9910459852837303
prefer_foot_left     : 0.009260754154407788


#### Stack models

In [23]:
top_12_models = regression.compare_models(sort='RMSE', n_select=12)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,1.502298e+05,4.100785e+11,5.938080e+05,0.9885,0.1400,0.0769,2.410
gbr,Gradient Boosting Regressor,2.272081e+05,5.397980e+11,7.054662e+05,0.9838,0.2892,0.1961,0.422
lightgbm,Light Gradient Boosting Machine,1.723816e+05,6.427013e+11,7.441760e+05,0.9822,0.1327,0.0799,0.166
et,Extra Trees Regressor,1.770660e+05,6.515007e+11,7.661427e+05,0.9807,0.1061,0.0565,1.473
rf,Random Forest Regressor,1.827794e+05,6.956995e+11,7.947449e+05,0.9800,0.1003,0.0579,1.652
xgboost,Extreme Gradient Boosting,1.781466e+05,8.075571e+11,8.131173e+05,0.9776,0.1141,0.0708,0.676
dt,Decision Tree Regressor,2.520780e+05,1.564889e+12,1.206346e+06,0.9530,0.1393,0.0677,0.035
ada,AdaBoost Regressor,1.841299e+06,4.670438e+12,2.146380e+06,0.8497,1.6054,5.3290,0.311
knn,K Neighbors Regressor,7.996177e+05,7.713132e+12,2.736010e+06,0.7632,0.4012,0.3371,0.070
llar,Lasso Least Angle Regression,1.729087e+06,1.045016e+13,3.203581e+06,0.6717,1.2607,4.4734,0.019


INFO:logs:create_model_container: 22
INFO:logs:master_model_container: 22
INFO:logs:display_container: 4
INFO:logs:[<catboost.core.CatBoostRegressor object at 0x7f7a19920b50>, GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False), LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
          

In [24]:
fe_dict.update(regression.pull()['RMSE'])

In [25]:
ctb2 = regression.create_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,144759.3753,1.520132e+11,3.898887e+05,0.9951,0.1419,0.0706
1,154180.7285,3.910186e+11,6.253148e+05,0.9865,0.1335,0.0790
2,129393.3281,1.778997e+11,4.217816e+05,0.9922,0.1719,0.0807
3,122332.5349,1.639591e+11,4.049186e+05,0.9938,0.1048,0.0698
4,157366.1067,6.301804e+11,7.938390e+05,0.9839,0.1566,0.0717
5,133201.9209,1.951577e+11,4.417666e+05,0.9928,0.1961,0.0884
6,180502.9608,4.262718e+11,6.528950e+05,0.9908,0.1441,0.0811
7,203429.5926,1.458906e+12,1.207852e+06,0.9702,0.1240,0.0760
8,144535.8020,3.053806e+11,5.526125e+05,0.9875,0.1186,0.0760


INFO:logs:create_model_container: 23
INFO:logs:master_model_container: 23
INFO:logs:display_container: 5
INFO:logs:<catboost.core.CatBoostRegressor object at 0x7f7a178808d0>
INFO:logs:create_model() succesfully completed......................................


In [26]:
%%time
for top_model in top_12_models[:5]:
  regression.tune_model(top_model, optimize='RMSE').get_best_score()['learn']['RMSE']

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,215642.9899,3.844132e+11,6.200107e+05,0.9877,0.1579,0.1100
1,197717.7572,2.783095e+11,5.275504e+05,0.9904,0.2324,0.1139
2,174746.1203,1.957848e+11,4.424758e+05,0.9914,0.2101,0.1239
3,167737.3815,2.389585e+11,4.888338e+05,0.9910,0.1811,0.1220
4,200088.0465,4.474700e+11,6.689320e+05,0.9886,0.1691,0.1189
5,188622.2448,2.717622e+11,5.213082e+05,0.9900,0.2727,0.1418
6,248229.5065,6.818037e+11,8.257141e+05,0.9853,0.2171,0.1191
7,253667.2979,1.043048e+12,1.021297e+06,0.9787,0.1884,0.1243
8,206127.7270,4.513184e+11,6.718024e+05,0.9815,0.1867,0.1286


INFO:logs:create_model_container: 25
INFO:logs:master_model_container: 25
INFO:logs:display_container: 7
INFO:logs:GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=6,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.3, min_impurity_split=None,
                          min_samples_leaf=4, min_samples_split=10,
                          min_weight_fraction_leaf=0.0, n_estimators=270,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=0.7, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
INFO:logs:tune_model() succesfully completed......................................


AttributeError: ignored

In [27]:
%%time
stacker = regression.stack_models(top_12_models[:5], fold=10, meta_model=regression.tune_model(ctb2, optimize='RMSE'))
regression.predict_model(stacker)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,204602.1019,3.779479e+11,6.147747e+05,0.9879,0.1561,0.1120
1,248315.8165,1.600037e+12,1.264926e+06,0.9447,0.1930,0.1371
2,191856.8629,3.429517e+11,5.856208e+05,0.9849,0.1888,0.1322
3,157779.1599,1.625114e+11,4.031270e+05,0.9939,0.1663,0.1130
4,187448.6586,3.170297e+11,5.630539e+05,0.9919,0.1754,0.1251
5,166800.7718,1.832088e+11,4.280290e+05,0.9933,0.1958,0.1319
6,268460.3001,1.104166e+12,1.050793e+06,0.9761,0.2156,0.1518
7,260942.1241,1.559033e+12,1.248613e+06,0.9682,0.1800,0.1235
8,219330.4425,1.305979e+12,1.142794e+06,0.9463,0.1677,0.1159


INFO:logs:create_model_container: 27
INFO:logs:master_model_container: 27
INFO:logs:display_container: 9
INFO:logs:StackingRegressor(cv=5,
                  estimators=[('catboost',
                               <catboost.core.CatBoostRegressor object at 0x7f7a157ca0d0>),
                              ('gbr',
                               GradientBoostingRegressor(alpha=0.9,
                                                         ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='ls', max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
  

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Stacking Regressor,207641.9264,6.090996e+11,780448.3569,0.9842,0.2156,0.1528


CPU times: user 5.77 s, sys: 1.09 s, total: 6.86 s
Wall time: 6min 3s


,age,continent,contract_until,position,reputation,stat_overall,stat_potential,stat_skill_moves,stat_poa,prefer_foot_left,value,Label
0,0.817833,4,0.250,2,0.00,0.425532,0.413043,1,0.214321,0.0,600000.0,8.229939e+05
1,-0.260856,4,0.625,2,0.00,0.425532,0.478261,2,-0.319196,0.0,975000.0,8.912679e+05
2,2.543736,2,0.375,1,0.00,0.531915,0.521739,0,2.080220,0.0,210000.0,4.623460e+05
3,-0.045118,2,0.125,0,0.00,0.425532,0.478261,1,-0.177195,0.0,800000.0,8.002914e+05
4,-0.692332,0,0.500,2,0.00,0.531915,0.673913,2,0.061572,0.0,4400000.0,3.980349e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
2675,0.386357,2,0.125,2,0.00,0.468085,0.456522,2,0.164847,0.0,1100000.0,9.129914e+05
2676,0.817833,2,0.125,3,0.25,0.531915,0.521739,2,0.824560,0.0,3100000.0,2.903575e+06
2677,-0.260856,4,0.125,3,0.00,0.382979,0.434783,1,-0.515393,1.0,675000.0,5.410401e+05
2678,-1.123808,2,0.000,3,0.00,0.276596,0.478261,2,-1.183916,0.0,325000.0,3.538228e+05


In [28]:
# 결과 모델 점수

fe_dict['catboost tuned'] = ctb_tuned.get_best_score()['learn']['RMSE']
fe_dict['stack5 tuned'] = regression.pull().at[0, 'RMSE']

### Blend Model

In [29]:
%%time
blended5 = regression.blend_models(top_12_models[:5], 10, optimize='RMSE')
regression.predict_model(blended5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,158742.5772,2.313965e+11,4.810369e+05,0.9926,0.0905,0.0600
1,167986.8533,5.965566e+11,7.723708e+05,0.9794,0.1430,0.0730
2,135413.2546,1.903177e+11,4.362542e+05,0.9916,0.1395,0.0674
3,139069.3566,2.588825e+11,5.088049e+05,0.9902,0.1067,0.0680
4,154316.9792,4.907055e+11,7.005038e+05,0.9875,0.1082,0.0673
5,142861.1275,2.892865e+11,5.378536e+05,0.9894,0.1644,0.0880
6,214747.9056,9.276102e+11,9.631252e+05,0.9800,0.2213,0.0728
7,222520.9814,1.467943e+12,1.211587e+06,0.9700,0.1268,0.0733
8,156717.2623,3.975268e+11,6.304973e+05,0.9837,0.1269,0.0756


INFO:logs:create_model_container: 28
INFO:logs:master_model_container: 28
INFO:logs:display_container: 11
INFO:logs:VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7f7a157f1610>),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                        

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,164681.8299,1.057754e+12,1.028471e+06,0.9726,0.144,0.0776


CPU times: user 1.93 s, sys: 499 ms, total: 2.43 s
Wall time: 59.8 s


,age,continent,contract_until,position,reputation,stat_overall,stat_potential,stat_skill_moves,stat_poa,prefer_foot_left,value,Label
0,0.817833,4,0.250,2,0.00,0.425532,0.413043,1,0.214321,0.0,600000.0,6.110029e+05
1,-0.260856,4,0.625,2,0.00,0.425532,0.478261,2,-0.319196,0.0,975000.0,9.266294e+05
2,2.543736,2,0.375,1,0.00,0.531915,0.521739,0,2.080220,0.0,210000.0,3.540435e+05
3,-0.045118,2,0.125,0,0.00,0.425532,0.478261,1,-0.177195,0.0,800000.0,7.780794e+05
4,-0.692332,0,0.500,2,0.00,0.531915,0.673913,2,0.061572,0.0,4400000.0,4.169551e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
2675,0.386357,2,0.125,2,0.00,0.468085,0.456522,2,0.164847,0.0,1100000.0,1.066482e+06
2676,0.817833,2,0.125,3,0.25,0.531915,0.521739,2,0.824560,0.0,3100000.0,2.913028e+06
2677,-0.260856,4,0.125,3,0.00,0.382979,0.434783,1,-0.515393,1.0,675000.0,6.626627e+05
2678,-1.123808,2,0.000,3,0.00,0.276596,0.478261,2,-1.183916,0.0,325000.0,3.532469e+05


In [30]:
fe_dict['blend5'] = regression.pull().at[0, 'RMSE']

In [33]:
joblib.dump(fe_dict, 'fe.pkl')

['fe.pkl']

In [34]:
fe_dict

{'catboost': 593807.9794,
 'gbr': 705466.1986,
 'lightgbm': 744176.0045,
 'et': 766142.7218,
 'rf': 794744.9264,
 'xgboost': 813117.2688,
 'dt': 1206345.7145,
 'ada': 2146380.1892,
 'knn': 2736010.25,
 'llar': 3203580.9981,
 'lasso': 3203593.775,
 'lr': 3203594.225,
 'ridge': 3203733.775,
 'lar': 3444259.1831,
 'omp': 4297419.608,
 'huber': 4402101.6159,
 'en': 4426573.5,
 'par': 5131785.0391,
 'dummy': 5623610.85,
 'br': 5623610.9108,
 'catboost tuned': 421487.8915681472,
 'stack5 tuned': 780448.3569,
 'blend5': 1028471.4549}